# MIT Movie Dataset - Explainability



In [ ]:
import pandas as pd
import numpy as np

# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.chdir('/content/drive/My Drive/Colab Notebooks/DAAN888/data')
#os.chdir('/content/drive/My Drive/DAAN888/data')
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/DAAN888/data'

In [ ]:
model_dir = '/content/drive/My Drive/Colab Notebooks/DAAN888/models/'

In [ ]:
data_dir = '/content/drive/My Drive/Colab Notebooks/DAAN888/data/'

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Nov  9 02:08:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------



---



## Load Dataset

In [ ]:
import pickle 

with open('mitmovie.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

In [ ]:
# first row in train set
list(zip(dataset['train_tokens'][0], dataset['train_labels'][0]))

[('what', 'O'),
 ('movies', 'O'),
 ('star', 'O'),
 ('bruce', 'B-ACTOR'),
 ('willis', 'I-ACTOR')]

## Bert Model

Documentation for this model can be found: 
https://huggingface.co/transformers/model_doc/bert.html#overview

Following HuggingFace distilbert tutorial: https://huggingface.co/transformers/custom_datasets.html#tok-ner


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 8.8MB/s 
     |████████████████████████████████| 890kB 30.4MB/s 
     |████████████████████████████████| 2.9MB 54.2MB/s 
     |████████████████████████████████| 1.1MB 48.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d01d05f455de50a3b748060cd0dbc6af3a7134a10eca25f6367799b934658e35
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Encode Labels

In [ ]:
# get the set of unique labels in the movie dataset
uniq_labels = set([label for doc in dataset['train_labels'] for label in doc])

In [ ]:
# assign a number to each label
label_encoding = {label: id for id, label in enumerate(uniq_labels)}
label_encoding

{'B-ACTOR': 21,
 'B-CHARACTER': 0,
 'B-DIRECTOR': 23,
 'B-GENRE': 20,
 'B-PLOT': 10,
 'B-RATING': 12,
 'B-RATINGS_AVERAGE': 19,
 'B-REVIEW': 11,
 'B-SONG': 24,
 'B-TITLE': 1,
 'B-TRAILER': 14,
 'B-YEAR': 5,
 'I-ACTOR': 7,
 'I-CHARACTER': 22,
 'I-DIRECTOR': 8,
 'I-GENRE': 9,
 'I-PLOT': 15,
 'I-RATING': 13,
 'I-RATINGS_AVERAGE': 18,
 'I-REVIEW': 3,
 'I-SONG': 6,
 'I-TITLE': 17,
 'I-TRAILER': 2,
 'I-YEAR': 16,
 'O': 4}

### Encode Texts

To encode the texts, we have to use the same Tokenizer that DistilBert was trained on.

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased')

In [ ]:
print('There are %s words in this tokenizer object' % tokenizer.vocab_size)

There are 30522 words in this tokenizer object


In [ ]:
# use the tokenizer to encode the texts 
train_encodings = tokenizer(dataset['train_tokens'], 
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding=True, 
                            truncation=True)

test_encodings = tokenizer(dataset['test_tokens'], 
                           is_split_into_words=True, 
                           return_offsets_mapping=True, 
                           padding=True, 
                           truncation=True)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])

In [ ]:
# make sure same number of docs
len(train_encodings['input_ids']), len(dataset['train_tokens'])

(9775, 9775)

In [ ]:
dataset['train_tokens'][0]

['what', 'movies', 'star', 'bruce', 'willis']

In [ ]:
# preview what the encoded result looks like
list(zip(train_encodings['input_ids'][0][0:12], train_encodings['attention_mask'][0][0:12], train_encodings['offset_mapping'][0][0:12]))

[(101, 1, (0, 0)),
 (2054, 1, (0, 4)),
 (5691, 1, (0, 6)),
 (2732, 1, (0, 4)),
 (5503, 1, (0, 5)),
 (12688, 1, (0, 6)),
 (102, 1, (0, 0)),
 (0, 0, (0, 0)),
 (0, 0, (0, 0)),
 (0, 0, (0, 0)),
 (0, 0, (0, 0)),
 (0, 0, (0, 0))]

In [ ]:
# the model expects all docs to be same length (51) 
# the attention mask will tell the model to ignore the padding with zeros
print('Length of sequences is %s ' % len(train_encodings['input_ids'][1]))

Length of sequences is 51 


In [ ]:
# first document in dataset
print( dataset['train_tokens'][0] )

# check out new tokenization result as words
print( tokenizer.convert_ids_to_tokens( train_encodings['input_ids'][0][0:9]) )

# check out new tokenization result as ids
print( [val for val in train_encodings['input_ids'][0] if val != 0] )

['what', 'movies', 'star', 'bruce', 'willis']
['[CLS]', 'what', 'movies', 'star', 'bruce', 'willis', '[SEP]', '[PAD]', '[PAD]']
[101, 2054, 5691, 2732, 5503, 12688, 102]


In [ ]:
find_subwords = []
for i, offset_list in enumerate(train_encodings['offset_mapping']):
  for j, offset_tuple in enumerate(offset_list):
    if offset_tuple[0] != 0:
      find_subwords.append(i)

In [ ]:
np.unique(find_subwords)

array([   1,    2,    3, ..., 9752, 9767, 9772])

In [ ]:
dataset['train_tokens'][1]

['show', 'me', 'films', 'with', 'drew', 'barrymore', 'from', 'the', '1980s']

In [ ]:
list(zip(train_encodings['input_ids'][1][0:12], train_encodings['attention_mask'][1][0:12], train_encodings['offset_mapping'][1][0:12]))

[(101, 1, (0, 0)),
 (2265, 1, (0, 4)),
 (2033, 1, (0, 2)),
 (3152, 1, (0, 5)),
 (2007, 1, (0, 4)),
 (3881, 1, (0, 4)),
 (6287, 1, (0, 5)),
 (5974, 1, (5, 9)),
 (2013, 1, (0, 4)),
 (1996, 1, (0, 3)),
 (3865, 1, (0, 5)),
 (102, 1, (0, 0))]

In [ ]:
[tokenizer.convert_ids_to_tokens(val) for val in train_encodings['input_ids'][1][0:12]]

['[CLS]',
 'show',
 'me',
 'films',
 'with',
 'drew',
 'barry',
 '##more',
 'from',
 'the',
 '1980s',
 '[SEP]']

### Adjust Labels for Vocab Offset


In the print out above you can see that "barrymore" gets transformed into "barry", "##more". This is because the name is not in the vocabulary of the model, however, those small subwords "barry" and "##more" are. So the offsetting notifies the model of this splitting of the word. However, we have to adjust the labels now to account for this separation. 

Based on https://datascience.stackexchange.com/questions/69640/what-should-be-the-labels-for-subword-tokens-in-bert-for-ner-task we will not drag the label to the new subword feature because that would introduce more instances of that class and change the number of support instances thus making the models difficult to compare. 

Also some comments to consider for an alternative strategy: https://github.com/google-research/bert/issues/646


In [ ]:
import numpy as np

def adjust_labels_for_offset(original_labels, label_dictionary, encodings):

    # convert to the numeric encoding of the label
    labels = [[label_dictionary[label] for label in doc] for doc in original_labels]

    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):

        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [ ]:
train_labels = adjust_labels_for_offset(dataset['train_labels'], 
                                        label_encoding, 
                                        train_encodings)

test_labels = adjust_labels_for_offset(dataset['test_labels'],
                                      label_encoding, 
                                      test_encodings)

In [ ]:
id_to_label = {id: label for (label,id) in label_encoding.items()}
id_to_label[-100] = 'X'

#print( tokenizer.convert_ids_to_tokens(encoding_example) )
#print([id_to_label[id] for id in train_labels[0][0:9]])
list(zip( tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][0][0:15]), [id_to_label[id] for id in train_labels[0][0:15]], train_labels[0][0:15] )) 

[('[CLS]', 'X', -100),
 ('what', 'O', 4),
 ('movies', 'O', 4),
 ('star', 'O', 4),
 ('bruce', 'B-ACTOR', 21),
 ('willis', 'I-ACTOR', 7),
 ('[SEP]', 'X', -100),
 ('[PAD]', 'X', -100),
 ('[PAD]', 'X', -100),
 ('[PAD]', 'X', -100),
 ('[PAD]', 'X', -100),
 ('[PAD]', 'X', -100),
 ('[PAD]', 'X', -100),
 ('[PAD]', 'X', -100),
 ('[PAD]', 'X', -100)]

In [ ]:
np.unique(train_labels)

array([-100,    0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
         10,   11,   12,   13,   14,   15,   16,   17,   18,   19,   20,
         21,   22,   23,   24])

### Prepare Pytorch Datasets

https://huggingface.co/transformers/custom_datasets.html#ft-trainer


In [ ]:
import torch

# pytorch is expecting a certain type of dataset 

class pt_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# remove the offset_mapping
train_encodings.pop("offset_mapping") 
test_encodings.pop("offset_mapping")

train_dataset = pt_dataset(train_encodings, train_labels)
test_dataset = pt_dataset(test_encodings, test_labels)

### Train Model

In [ ]:
from transformers import BertForTokenClassification

# load the pretrained model from huggingface
model = BertForTokenClassification.from_pretrained('bert-large-uncased', num_labels=len(uniq_labels))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large

In [ ]:
from transformers import Trainer, TrainingArguments

# boiler plate code from huggingface to launch a trainer instance
# sets directories and baseline configuration for batch sizes and weight decay

training_args = TrainingArguments(
    output_dir = model_dir +  'explainability/bert',          # output directory
    overwrite_output_dir = True,
    num_train_epochs=3,              # total number of training epochs
    evaluation_strategy = 'epoch',
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'explainability/bert/logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset = train_dataset,         # training dataset
    eval_dataset = test_dataset             # evaluation dataset
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [ ]:
model.to(device);

In [ ]:
from datetime import datetime

start = datetime.now()

trainer.train()

print('Time to train:', datetime.now() - start)

Epoch,Training Loss,Validation Loss
1,0.257758,0.285916
2,0.220679,0.252612
3,0.127478,0.241965


Time to train: 0:15:44.144982


In [ ]:
#trainer.save_model(model_dir + 'mitmovie_pt_bert_uncased/model')
#model.save_pretrained(model_dir + 'mitmovie_pt_bert_uncased/model')

### Interpret Predictions

Guidance for how to do interpretability with NER was taken from: 
https://www.depends-on-the-definition.com/interpretable-named-entity-recognition/


In [ ]:
# from transformers import BertForTokenClassification
# import torch

# # retreive the saved model 
# model = BertForTokenClassification.from_pretrained(model_dir + 'mitmovie_pt_bert_uncased/model', num_labels=len(uniq_labels))

In [ ]:
# from transformers import BertTokenizerFast

# tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased')

In [ ]:
!pip install eli5
!pip install lime
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler

In [ ]:
model.to('cpu'); # push model to cpu

In [ ]:
id_to_label = {id: label for (label,id) in label_encoding.items()}

The function below will yield probabilities for a specific word in the sentence you want to interpret.

In [ ]:
def make_prediction(texts):

  # if type(text) == str:
  #   text = text
  # if type(text) == tuple:
  #   text = text[0]

  texts = [text.split() for text in texts]

  encodings = tokenizer(texts,  # [text.split()]
            is_split_into_words=True, 
            return_offsets_mapping=False, 
            padding=True, 
            truncation=True)
  
  with torch.no_grad():
      output = model( torch.tensor(encodings['input_ids']) )

  preds = output[0]  
  preds = torch.softmax(preds, dim = -1) # get probabilities

  return preds[:,word_to_check,:].numpy() # return probs only for the word you want

Assign the sentence and the word to inspect


In [ ]:
sentence =  ' '.join( dataset['train_tokens'][6] ) # assign by index value [#]
sentence

'show me science fiction films directed by steven spielberg'

Figure out which index the word you want to predict is in. Some tokens get split into subwords making more tokens than original. Also, tokenizer will add a [CLS] at beginning and a [SEP] token at end of every sentence.

In [ ]:
# encode the sentence
encodings = tokenizer([sentence.split()],  # [text.split()]
            is_split_into_words=True, 
            return_offsets_mapping=False, 
            padding=True, 
            truncation=True)

# get tokenized version
tokes = tokenizer.convert_ids_to_tokens(encodings['input_ids'][0])

# get predictions
with torch.no_grad():
    output = model(torch.tensor(encodings['input_ids']))

# convert predictions to classes
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
predicted = [id_to_label[label] for label in label_indices.tolist()[0]]

# print the index, token, class                                        
for i,t in enumerate(zip(tokes, predicted)) :
  print(i, t[0], t[1])

0 [CLS] O
1 show O
2 me O
3 science B-GENRE
4 fiction I-GENRE
5 films O
6 directed O
7 by O
8 steven B-DIRECTOR
9 spielberg I-DIRECTOR
10 [SEP] I-DIRECTOR


In [ ]:
# assign the index of the token you want to inspect
word_to_check = 4 

In [ ]:
# checking output of instance to make sure we get back a prediction
id_to_label[np.argmax(make_prediction([sentence]), -1)[0]]

'I-GENRE'

In [ ]:
sampler = MaskingTextSampler(
    replacement = "UNK",
    max_replace = 0.7,
    bow = False # bag of words means it will test the exclusion of multiple words as well
)

In [ ]:
# illustrations of token removal for testing
samples, similarity = sampler.sample_near(sentence, n_samples=4)
for sample in samples:
  print(sample)
  print()
print('Similarities:', similarity)

show UNK UNK UNK UNK UNK UNK UNK spielberg

UNK me UNK UNK films UNK by UNK UNK

show me UNK fiction UNK UNK by UNK UNK

show me UNK UNK films directed UNK steven spielberg

Similarities: [0.47140438 0.57735014 0.66666655 0.81649647]


In [ ]:
text_explainer = TextExplainer(
    sampler=sampler,
    position_dependent = True,
    random_state = 35
)

text_explainer.fit(sentence, make_prediction)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


TextExplainer(char_based=False,
              clf=SGDClassifier(alpha=0.001, average=False, class_weight=None,
                                early_stopping=False, epsilon=0.1, eta0=0.0,
                                fit_intercept=True, l1_ratio=0.15,
                                learning_rate='optimal', loss='log',
                                max_iter=1000, n_iter_no_change=5, n_jobs=None,
                                penalty='elasticnet', power_t=0.5,
                                random_state=RandomState(MT19937) at 0x7F7FE87DC678,
                                shuffle=True, tol=0.001,
                                validation_fraction=0.1, verbose=0,
                                warm_start=False),
              expand_factor=10, n_samples=5000, position_dependent=True,
              random_state=35, rbf_sigma=None,
              sampler=MaskingTextSampler(bow=False, group_size=1,
                                         max_replace=0.7, min_replace=1,
         

In [ ]:
text_explainer.explain_prediction(
    target_names=list(label_encoding.keys()),
    top_targets=3 # will give you result for the 3 most probable
)

Explanation(estimator="SGDClassifier(alpha=0.001, average=False, class_weight=None,\n              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,\n              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,\n              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',\n              power_t=0.5, random_state=RandomState(MT19937) at 0x7F7FE87DC780,\n              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,\n              warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='I-GENRE', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[1] me', weight=4.466628009327518, std=None, value=1.0), FeatureWeight(feature='[0] show', weight=4.277139328832432, std=None, value=1.0), FeatureWeight(feature='[2] science', weight=3.8125296594473896, std=None, value=1.0), FeatureWeight(feature='[3] fiction', weight=3.756146119540561, std=None, value=1.0), FeatureWeight(feature='[4] films', weight=0.06843884952954969, std=None, value=1.0), FeatureWeight(feature='[6] by', weight=0.02175256833035516, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-14.513802360900472, std=None, value=1.0), FeatureWeight(feature='[5] directed', weight=-0.08531325933340446, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.5686191685854517, score=1.80351891477393, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='show me science fiction films directed by steven spielberg', spans=[('show', [(0, 4)], 4.277139328832432), ('me', [(5, 7)], 4.466628009327518), ('science', [(8, 15)], 3.8125296594473896), ('fiction', [(16, 23)], 3.756146119540561), ('films', [(24, 29)], 0.06843884952954969), ('directed', [(30, 38)], -0.08531325933340446), ('by', [(39, 41)], 0.02175256833035516)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=16.317321275674402, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-14.513802360900472, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='B-GENRE', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[2] science', weight=3.4691134123325673, std=None, value=1.0), FeatureWeight(feature='[3] fiction', weight=2.461717441372714, std=None, value=1.0), FeatureWeight(feature='[5] directed', weight=0.5639145117118358, std=None, value=1.0), FeatureWeight(feature='[4] films', weight=0.5168870479333563, std=None, value=1.0), FeatureWeight(feature='[8] spielberg', weight=0.3290086566021752, std=None, value=1.0), FeatureWeight(feature='[6] by', weight=0.31557493992659524, std=None, value=1.0), FeatureWeight(feature='[7] steven', weight=0.24446416870208648, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-6.168066793589745, std=None, value=1.0), FeatureWeight(feature='[1] me', weight=-1.3711706646050992, std=None, value=1.0), FeatureWeight(feature='[0] show', weight=-0.7629604530386646, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.26554011867855887, score=-0.40151773265217816, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='show me science fiction films directed by steven spielberg', spans=[('show', [(0, 4)], -0.7629604530386646), ('me', [(5, 7)], -1.3711706646050992), ('science', [(8, 15)], 3.4691134123325673), ('fiction', [(16, 23)], 2.461717441372714), ('films', [(24, 29)], 0.5168870479333563), ('directed', [(30, 38)], 0.5639145117118358), ('by', [(39, 41)], 0.31557493992659524), ('steven', [(42, 48)], 0.24446416870208648), ('spielberg', [(49, 58)], 0.3290086566021752)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=5.766549060937567, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-6.168066793589

In [ ]:
#!pip install git+https://github.com/sofiaherrero/lime-ner